# Data of temperatures in California

Author: Hugo Carrillo

Creation: July 2024

This notebook shows the very beginning steps in heatwave detection from meteorological stations data. 
1. Detection of missing data: quantification and cleaning.
2. Selection of stations. 
3. We generate cleaned data for heatwave detections.

## Libraries import

In [1]:
import numpy as np
import pandas as pd


from datetime import datetime
from calendar import monthrange

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) # Suppress specific RuntimeWarnings

In [2]:
%load_ext autoreload
%autoreload 2

# Parameters


In [3]:
METADATA_PATH_AND_FILENAME = 'data/original/cimis_metadata.csv'
TEMP_DATA_PATH = 'data/original/'
CLEANED_DATA_PATH = 'data/cleaned/'
MY_FUNCTIONS_PATH = '../functions/'


# Utilities


In [7]:
def Month_Days(year):
    return {"year": year, "month_days": {m: monthrange(year, m)[1] for m in range(1, 13)}}

def Tfilter(data, column_label, nperc, year_window_init: int, year_window_end: int):
    start_date = data.index[0]
    end_date = data.index[-1]
    perc_label = 'perc'
    Tadd = 0.0
    year_window_init = year_window_init
    year_window_end = year_window_end
    data_temp = data[column_label]

    data_threshold = pd.DataFrame(
        [],
        columns=[perc_label],
        index = data.index
    )
    for year in range(start_date.year, end_date.year + 1):
        month_days = Month_Days(year)
        for month in month_days["month_days"]:
            for day in range(1, month_days["month_days"][month] + 1):
                try:
                    current_date = datetime(year, month, day)
                except ValueError:
                    if current_date == start_date:
                        current_date = datetime(year, month, day+1)
                    if current_date == end_date:
                        current_date = datetime(year, month, day-1)
                if  current_date >= start_date and current_date <= end_date:
                    f_data_temp = data_temp[
                        (year_window_init <= data_temp.index.year)
                        & (data_temp.index.year <= year_window_end)
                        & (data_temp.index.day == day)
                        & (data_temp.index.month == month)
                    ]
                    try:
                        data_threshold.loc[datetime(year, month, day), perc_label] = f_data_temp.quantile(
                            nperc*0.01, interpolation="midpoint"
                        ).values[0]
                    except AttributeError:
                        data_threshold.loc[datetime(year, month, day), perc_label] = f_data_temp.quantile(
                            nperc*0.01, interpolation="midpoint"
                        )
                    except ValueError:
                        data_threshold.loc[datetime(year, month, day-1), perc_label] = f_data_temp.quantile(
                            nperc*0.01, interpolation="midpoint"
                        ).values[0] + Tadd

        smoothed_series = data_threshold.rolling(window=31, center=True).mean()
    return smoothed_series + Tadd
    #return data_threshold + Tadd

def to_format(data, max_temp_lim = 50, add_filter_year = None, filter_by_hist = False, filter = True, dropnans = True):

    data["Date"] = pd.to_datetime(data["Date"],format="%Y-%m-%d")

    data = data.rename(columns={"Date":"date"})
    if dropnans:
        max_temp = data.set_index("date").dropna(subset=["DayAirTmpMax"])[["DayAirTmpMax"]]
        min_temp = data.set_index("date").dropna(subset=["DayAirTmpMin"])[["DayAirTmpMin"]]
        mean_temp = data.set_index("date").dropna(subset=["DayAirTmpAvg"])[["DayAirTmpAvg"]]
    else:
        max_temp = data.set_index("date")[["DayAirTmpMax"]]
        min_temp = data.set_index("date")[["DayAirTmpMin"]]
        mean_temp = data.set_index("date")[["DayAirTmpAvg"]]

    max_temp = max_temp.rename(columns={"DayAirTmpMax":"max_temp"})
    min_temp = min_temp.rename(columns={"DayAirTmpMin":"min_temp"})
    mean_temp = mean_temp.rename(columns={"DayAirTmpAvg":"mean_temp"})

    if filter:

        if add_filter_year is None:
            max_temp = max_temp.drop(max_temp[np.abs(max_temp["max_temp"])>max_temp_lim].index)
        else:
        #if add_filter_year is not None:
            max_temp.loc[(max_temp.index.year < add_filter_year) & (np.abs(max_temp['max_temp']) > max_temp_lim), 'max_temp'] = np.nan
            max_temp = max_temp.dropna(subset=["max_temp"])[["max_temp"]]


        if filter_by_hist:#add_filter_year is not None:
            perc_max = Tfilter(max_temp, 'max_temp', 99.9, add_filter_year, 2023)
            perc_max = perc_max.reindex(max_temp.index)


            max_temp.loc[(max_temp.index.year < add_filter_year) & (max_temp['max_temp'] > perc_max['perc'] + 10), 'max_temp'] = np.nan

        if dropnans:
            max_temp = max_temp.dropna(subset=["max_temp"])[["max_temp"]]
            min_temp = min_temp.dropna(subset=["min_temp"])[["min_temp"]]
            mean_temp = mean_temp.dropna(subset=["mean_temp"])[["mean_temp"]]

    concatenated_df = pd.concat([max_temp, min_temp, mean_temp], axis=1)

    return concatenated_df

# Reading data

In [15]:
# get all cimis stations information
stations = pd.read_csv(METADATA_PATH_AND_FILENAME)
stations.head()

,StationNbr,Name,City,RegionalOffice,County,ConnectDate,DisconnectDate,IsActive,IsEtoStation,Elevation,GroundCover,HmsLatitude,HmsLongitude,ZipCodes,SitingDesc
0,1,Fresno/F.S.U. USDA,Fresno,South Central Region Office,Fresno,6/7/1982,9/25/1988,False,True,340,Grass,36º48'52N / 36.814444,-119º43'54W / -119.731670,"93766, 93762, 93761, 93760, 93759, 93755, 9375...",NaN
1,2,FivePoints,Five Points,South Central Region Office,Fresno,6/7/1982,12/31/2050,True,True,285,Grass,36º20'10N / 36.336222,-120º6'46W / -120.112910,93624,NaN
2,3,Beach /Santa Cruz CO,Watsonville,South Central Region Office,Santa Cruz,5/30/1982,8/25/1986,False,True,10,Grass,36º52'50N / 36.880556,-121º47'36W / -121.793330,"95077, 95076, 95075, 95019, 95018",NaN
3,4,Webb /Santa Cruz CO,Watsonville,South Central Region Office,Santa Cruz,5/30/1982,4/29/1988,False,True,230,Grass,36º58'21N / 36.9725,-121º43'34W / -121.726110,"95077, 95076, 95075, 95019, 95018",NaN
4,5,Shafter,Shafter,South Central Region Office,Kern,6/1/1982,12/31/2050,True,False,360,Grass,35º31'57N / 35.532556,-119º16'54W / -119.281790,"93263, 93280, 93388",NaN


In [16]:
# Reading raw data of selected stations
statlist = [43, 5, 91, 90, 70, 62, 52, 47, 39, 35, 6]

stations_data_no_filter = {}
stations_data_filter_nans = {}
stations_data_filter1 = {}
stations_data_filter2 = {}
dropnans = False

for stat in statlist:
    print(stat)
    station_data_to_read = pd.read_excel(TEMP_DATA_PATH + 'Stat_' + str(stat) + '.xlsx')
    stations_data_no_filter[stat] = to_format(station_data_to_read, max_temp_lim=50, add_filter_year=None, filter_by_hist = False, filter = False, dropnans=dropnans)
    stations_data_filter_nans[stat] = to_format(station_data_to_read, max_temp_lim=50, add_filter_year=None, filter_by_hist = False, filter = False, dropnans=dropnans)
    stations_data_filter1[stat] = to_format(station_data_to_read, max_temp_lim=50, add_filter_year=None, filter_by_hist = False, filter = True, dropnans=dropnans)
    if stat == 70 or stat == 52:
        stations_data_filter2[stat] = to_format(station_data_to_read, max_temp_lim=50, add_filter_year=2008, filter_by_hist = True, dropnans=dropnans)
    else:
        stations_data_filter2[stat] = to_format(station_data_to_read, max_temp_lim=50, add_filter_year=None, filter_by_hist = False, dropnans=dropnans)

    #save
    stations_data_filter2[stat].to_parquet(CLEANED_DATA_PATH + f'Stat_{stat}.parquet')


43
5
91
90
70
62
52
47
39
35
6


In [9]:
#Percentage of missing and cleaned data

df_nans_and_deleted = pd.DataFrame(index=statlist)
df_total_days = {}
df_filter_nan_days = {}
df_filter1_days = {}
d_filter2_days = {}
total_days = np.zeros((len(statlist),))
filter_nan_days = np.zeros((len(statlist),))
filter1_days = np.zeros((len(statlist),))
filter2_days = np.zeros((len(statlist),))

for i, stat in enumerate(statlist):
    df_total_days = stations_data_no_filter[stat][stations_data_no_filter[stat].index.year>1988]['max_temp']
    #np.isnan(station_data_to_read[stat]['temperature'])
    df_filter_nan_days = stations_data_filter_nans[stat][stations_data_filter_nans[stat].index.year>1988]['max_temp']
    df_filter1_days = stations_data_filter1[stat][stations_data_filter1[stat].index.year>1988]['max_temp']
    df_filter2_days = stations_data_filter2[stat][stations_data_filter2[stat].index.year>1988]['max_temp']

    #station_data_to_read[stat][np.isnan(station_data_to_read[stat]['temperature'])]
    total_days[i] = len(df_total_days)#[np.isnan(df_total_days)])
    filter_nan_days[i] = len(df_filter_nan_days[np.isnan(df_total_days)])
    filter1_days[i] = len(df_filter1_days[np.isnan(df_filter1_days)])
    filter2_days[i] = len(df_filter2_days[np.isnan(df_filter2_days)])

df_nans_and_deleted['total'] = np.array(total_days)
df_nans_and_deleted['nans'] = np.array(filter_nan_days)/(np.array(total_days)[0])
df_nans_and_deleted['>50'] = -np.array(filter_nan_days)/(np.array(total_days)[0])+np.array(filter1_days)/(np.array(total_days)[0])
df_nans_and_deleted['filter2'] = np.array(filter2_days)/(np.array(total_days)[0]) - np.array(filter1_days)/(np.array(total_days)[0])


print(df_nans_and_deleted)

      total      nans       >50   filter2
43  13002.0  0.001615  0.000000  0.000000
5   12561.0  0.006076  0.000000  0.000000
91  12914.0  0.003076  0.000846  0.000000
90  12890.0  0.007383  0.001077  0.000000
70  13002.0  0.002538  0.010306  0.000692
62  13002.0  0.024919  0.001384  0.000000
52  13002.0  0.008537  0.006845  0.002230
47  12814.0  0.009229  0.000000  0.000000
39  13002.0  0.002769  0.000000  0.000000
35  13002.0  0.008460  0.000000  0.000000
6   13002.0  0.002538  0.000000  0.000000


In [10]:
print(r'$\frac{total missing data}{total days} =$' +
      str(np.sum(df_nans_and_deleted['nans']*df_nans_and_deleted['total'])/np.sum(df_nans_and_deleted['total'])))

print(r'$\frac{total missing data}{total days} =$' +
      str(np.sum(df_nans_and_deleted['>50']*df_nans_and_deleted['total'])/np.sum(df_nans_and_deleted['total'])))

$\frac{total missing data}{total days} =$0.007015026718621114
$\frac{total missing data}{total days} =$0.0018693251642777136
